In [1]:
from bert_embedding import BertEmbedding

# bert_abstract = """We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers.
#  Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.
#  As a result, the pre-trained BERT representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications. 
# BERT is conceptually simple and empirically powerful. 
# It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE benchmark to 80.4% (7.6% absolute improvement), MultiNLI accuracy to 86.7 (5.6% absolute improvement) and the SQuAD v1.1 question answering Test F1 to 93.2 (1.5% absolute improvement), outperforming human performance by 2.0%."""

# sentences = bert_abstract.split('\n')
# bert_embedding = BertEmbedding()
# result = bert_embedding(sentences)

: 

: 

In [2]:
print("hello")

hello


In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary-multilingual)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/Users/paulberard/opt/anaconda3/envs/projet-anlp/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 995526/995526 [00:00<00:00, 2008497.95B/s]


In [8]:
text = "I am so happy, today is my birthday! I can't wait to dance with my friends!"
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)
segments_ids = [1] * len(tokenized_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Print out the tokens.
print(tokenized_text)

['[CLS]', 'I', 'am', 'so', 'happy', ',', 'today', 'is', 'my', 'birthday', '!', 'I', 'can', "'", 't', 'wait', 'to', 'dance', 'with', 'my', 'friends', '!', '[SEP]']


In [9]:

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [3]:
# Load pre-trained model (weights)
#model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
#model.eval()

100%|██████████| 662804195/662804195 [01:53<00:00, 5847237.62B/s] 


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Li

In [10]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(encoded_layers, dim=0)

# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([23, 12, 768])

In [11]:

# `encoded_layers` has shape [12 x 1 x 23 x 768]

# `token_vecs` is a tensor with shape [23 x 768]
token_vecs = encoded_layers[11][0]

# Calculate the average of all 23 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [12]:
print("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


In [13]:
sentence_embedding

tensor([-6.1597e-02, -4.1800e-01, -1.6561e-02,  1.7259e-01,  6.4464e-01,
         2.2431e-01,  3.5428e-03, -2.0353e-01, -1.7712e-01,  1.9254e-01,
         6.3306e-02,  4.9740e-01,  2.8774e-01,  1.3299e-01,  4.8750e-01,
        -2.4759e-01,  6.0300e-01, -3.3866e-01,  3.6454e-02, -9.3834e-02,
         4.5473e-02,  4.5309e-02,  7.9374e-02,  2.0791e-03,  1.2832e-01,
         6.7566e-01, -3.5628e-01, -3.9083e-03, -1.5017e-01, -5.3016e-01,
         3.3515e-02,  5.1584e-01, -3.1449e-01,  8.3520e-01, -1.4942e-02,
         3.1487e-01, -3.4166e-02,  4.8147e-02,  2.3929e-01, -6.6266e-02,
        -4.6305e-02,  2.4869e-01,  1.9579e-01, -2.0513e-01,  1.8091e-02,
        -3.7220e-01,  4.7565e-01,  4.8067e-01, -2.3334e-02, -1.6669e-01,
         1.3784e-01, -1.6271e-01,  5.7715e-01,  2.7957e-01,  1.1482e-01,
         4.0967e-01, -1.1780e-02,  2.2493e-02,  3.6824e-01, -1.2241e-01,
         1.1530e-01,  1.1278e-01,  4.0873e-01, -5.8943e-01, -3.8034e-01,
         2.0094e-01, -1.5384e-02, -6.2539e-03,  1.0